In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("grassknoted/asl-alphabet")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
IMG_SIZE = 64
BATCH_SIZE = 32

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split = 0.2
)

In [ ]:
train_data = datagen.flow_from_directory(
    path,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    path,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    subset='validation'
)

# Khai báo mô hình - fit

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(64,64,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(train_data.num_classes, activation="softmax")
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentrophy',
    metrics=['accuracy']
)

history = model.fit(
    train_data,
    epochs=10,
    validation_data = val_data,
)

model.summary()

# Lưu mô hình

In [ ]:
model.save('asl_alphabet_model.h5')
print("Saved model")

# Đánh giá mô hình

In [ ]:
loss, accuracy = model.evaluate(val_data)
print(f"Accuracy: {accuracy:.4f}")
print(f"Loss: {loss:.4f}")

# Dự đoán

In [ ]:
loaded_model = keras.model.load_model('asl_alphabet_model.h5')
print("Loaded model from disk")

In [ ]:
class_name = list(train_data.class_indicies.keys())
print(class_name)

In [ ]:
img_path = path+'/test.jpg'
img = image.load_img(img_path, target_size = (IMG_SIZE, IMG_SIZE))
img_array = image.image_to_array()
img_array = img_array/255.0
img_array = np.expand_dims(img_array, axis=0)

In [ ]:
prediction = loaded_model.predict(img_array)
predicted_idex = np.argmax(prediction)
predicted_label = class_name[predicted_idex]

print(prediction)
print(f"Predicted label: {predicted_label}")